### Remove clouds from sentinel-2 bands

In [ ]:
folder = '/mnt/disk1/jasp/a2autocls2023/imgmad/step2_2021/'

bands = [
    'b02', 'b03', 'b04', 'b05', 'b06', 'b07',
    'b08', 'b8a', 'b09', 'b11', 'b12'
]

scl = 'scl'

ff = '.tif'

ofolder = '/mnt/disk1/jasp/a2autocls2023/imgmad/step3_2021/'

clouds_file = '/mnt/disk1/jasp/a2autocls2023/imgmad/clouds_2021.tif'

In [ ]:
import os
from glass.pys.oss  import lst_ff, fprop
from glass.pys.tm   import now_as_str
from glass.wenv.grs import run_grass
from glass.rst.rcls import rcls_rules

In [ ]:
# list bands
imgs = lst_ff(folder, file_format=ff, fnpart=bands)

# List scl
scls = lst_ff(folder, file_format=ff, fnpart=[scl])

In [ ]:
# Create GRASS GIS Session

ws, loc = ofolder, f"loc_{now_as_str()}"

grsb = run_grass(ws, location=loc, srs=imgs[0])
    
import grass.script.setup as gsetup
    
gsetup.init(grsb, ws, loc, 'PERMANENT')

In [ ]:
from glass.it.rst   import rst_to_grs, grs_to_rst
from glass.rst.rcls import rcls_rst, set_null, null_to_value
from glass.rst.alg  import rstcalc

In [ ]:
cldrules = rcls_rules({
    0 : 'NULL',
    1 : 'NULL',
    2 : 1,
    3 : 1,
    4 : 'NULL',
    5 : 'NULL',
    6 : 'NULL',
    7 : 'NULL',
    8 : 1,
    9 : 1,
    10 : 1,
    11 : 'NULL'
}, os.path.join(ws, loc, 'clouds_rules.txt'))

rscl = []
for s in scls:
    # Add SCL to GRASS GIS
    _s = rst_to_grs(s, fprop(s, 'fn'))

    # Reclassify all files to get only things related to clouds
    rs = rcls_rst(_s, cldrules, f'rcls_{_s}', api='grass')
    _rs = rstcalc(rs, f'cp_{_s}', api='grass')

    # Clouds to NoData
    null_to_value(_rs, 0, as_cmd=True)
    set_null(_rs, 1, ascmd=True)
    
    rscl.append(_rs)

# One file with all clouds as nodata
cloud_rst = rstcalc(" + ".join(rscl), 'clouds', api='grass')

# Export clouds
rcld = grs_to_rst(cloud_rst, clouds_file, is_int=True)

In [ ]:
for i in imgs:
    # Import all bands to GRASS GIS
    _i = rst_to_grs(i, fprop(i, 'fn'))

    # Sum to clouds file
    i_noclouds = rstcalc(f"{_i} + {cloud_rst}", f'nocld_{_i}', api='grass')

    # Export file
    grs_to_rst(i_noclouds, os.path.join(
        ofolder, os.path.basename(i)
    ), is_int=True)